In [1]:
from bs4 import BeautifulSoup
import urllib2
import pandas as pd
import numpy as np

from pandas_datareader import data, wb

from datetime import date, timedelta, datetime
from time import sleep
import os.path
import re
import random

import matplotlib.pyplot as plt
%matplotlib inline

from nltk.sentiment.vader import SentimentIntensityAnalyzer

from sentimenter import data as sn_data

In [2]:
from sklearn.linear_model import LinearRegression
# from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import StratifiedKFold, cross_val_score, train_test_split
from sklearn.metrics import explained_variance_score, mean_absolute_error, silhouette_score
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor

import pydot
from IPython.display import Image

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
companies = sn_data.get_companies()

In [4]:
companies['Ticker_symbol'] = companies['Ticker_symbol'].apply(lambda x: x.replace('.N', ''))

In [5]:
companies.head()

,Ticker_symbol,Security,SEC_filings,GICS_Sector,GICS_Sub_Industry,Address_of_Headquarters,Date_first_added,CIK
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,66740
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152
3,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373
4,ATVI,Activision Blizzard,reports,Information Technology,Home Entertainment Software,"Santa Monica, California",2015-08-31,718877


In [6]:
companies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 505 entries, 0 to 504
Data columns (total 8 columns):
Ticker_symbol              505 non-null object
Security                   505 non-null object
SEC_filings                505 non-null object
GICS_Sector                505 non-null object
GICS_Sub_Industry          505 non-null object
Address_of_Headquarters    505 non-null object
Date_first_added           305 non-null object
CIK                        505 non-null int64
dtypes: int64(1), object(7)
memory usage: 35.5+ KB


## Work on this for multiprocessing

In [ ]:
# import multiprocessing as mp

# filename = "datasets/articles.csv"
# chunksize = 100 # processing 100 rows at a time


# def process_frame(df):
#     # process data frame
#     sid = SentimentIntensityAnalyzer()
#     lambda_sentimenter = lambda x: pd.Series(
#         [sid.polarity_scores(x['Article'])['compound'],
#          sid.polarity_scores(x['Article'])['neg'],
#          sid.polarity_scores(x['Article'])['neu'],
#          sid.polarity_scores(x['Article'])['pos']
#         ])
#     return len(df)

# def get_sentimentaized_articles(companies):
#     reader = pd.read_table(filename, chunksize=chunksize)
#     pool = mp.Pool(6) # use 4 processes

#     funclist = []
    
#     for df in reader:
#             # process each data frame
#             f = pool.apply_async(process_frame,[df])
#             funclist.append(f)

#     result = 0
#     for f in funclist:
#             result += f.get(timeout=10) # timeout in 10 seconds

#     print "There are %d rows of data"%(result)


In [11]:
s_articles = pd.read_csv('datasets/articles_part_1.csv', encoding='utf8', index_col=0)
pd.concat([s_articles, pd.read_csv('datasets/articles_part_2.csv', encoding='utf8', index_col=0)], axis=0)

,Article,Date,Symbol,Time,Title,Url,compound,neg,neu,pos
0,MOSCOW Jan 17 Russia's biggest home electron...,2014-01-17,MMM,"Fri Jan 17, 2014 | 7:49am EST",Russia's M.video says consumer confidence weak...,http://www.reuters.com//article/russia-mvideo-...,0.0516,0.049,0.901,0.051
1,By Lewis Krauskopf Diversified manufacture...,2014-01-30,MMM,"Thu Jan 30, 2014 | 9:44am EST","3M revenue growth slows, misses Wall Street ta...",http://www.reuters.com//article/us-3m-results-...,0.7096,0.037,0.896,0.067
2,By Lewis KrauskopfJan 30 Diversified manufac...,2014-01-30,MMM,"Thu Jan 30, 2014 | 9:42am EST","UPDATE 2-3M revenue growth slows, misses Wall ...",http://www.reuters.com//article/3m-results-idU...,0.7964,0.038,0.886,0.076
3,Jan 30 Diversified manufacturer 3M Co poste...,2014-01-30,MMM,"Thu Jan 30, 2014 | 7:39am EST",Manufacturer 3M posts 11 percent rise in net i...,http://www.reuters.com//article/3m-results-idU...,0.7430,0.000,0.885,0.115
4,3M Co. has invested US$15 million in one of ...,2014-03-10,MMM,"Sun Mar 9, 2014 | 8:19pm EDT",SINGAPORE PRESS-3M to lift N95 mask output at ...,http://www.reuters.com//article/singapore-pres...,0.4019,0.000,0.958,0.042
5,March 18 Diversified manufacturer 3M Co exp...,2014-03-18,MMM,"Tue Mar 18, 2014 | 12:33am EDT",3M sees sales growth in China to triple global...,http://www.reuters.com//article/3m-china-sales...,0.7355,0.031,0.909,0.060
6,"MOSCOW, March 27 Russian home electronics re...",2014-03-27,MMM,"Thu Mar 27, 2014 | 3:18am EDT",Russia's M.video 2013 net profit rises 38 pct,http://www.reuters.com//article/russia-mvideo-...,0.6124,0.000,0.926,0.074
7,Post-it notes maker 3M Co (MMM.N) said its b...,2014-04-15,MMM,"Tue Apr 15, 2014 | 12:23pm EDT",3M board opposes proposal for shareholder acti...,http://www.reuters.com//article/us-3m-sharehol...,0.8271,0.000,0.902,0.098
8,April 15 Post-it notes maker 3M Co said its ...,2014-04-15,MMM,"Tue Apr 15, 2014 | 12:20pm EDT",3M board opposes proposal for shareholder acti...,http://www.reuters.com//article/3m-shareholder...,0.8271,0.000,0.903,0.097
9,* Apple rallies a day after announcing 7-for...,2014-04-24,MMM,"Thu Apr 24, 2014 | 4:40pm EDT",US STOCKS-Apple buoys Nasdaq; Ukraine weighs o...,http://www.reuters.com//article/markets-usa-st...,0.9617,0.055,0.851,0.094


In [ ]:
def update_sentimentalized_articles(articles):
    s_articles = pd.read_csv('datasets/articles.csv', encoding='utf8', index_col=0)
    print articles.shape, s_articles.shape
    not_s_syms = [x for x in list(articles.Symbol.unique()) if x not in list(s_articles.Symbol.unique())]
    print "need to sentimentalize {} symbols".format(len(not_s_syms))
    
    if len(not_s_syms) > 0:
        not_s_articles = articles[articles['Symbol'].isin(not_s_syms)]
        sid = SentimentIntensityAnalyzer()
        lambda_sentimenter = lambda x: pd.Series(
            [sid.polarity_scores(x['Article'])['compound'],
             sid.polarity_scores(x['Article'])['neg'],
             sid.polarity_scores(x['Article'])['neu'],
             sid.polarity_scores(x['Article'])['pos']
            ])
        sents = not_s_articles.apply(lambda_sentimenter, axis=1)
        sents.columns = ['compound', 'neg', 'neu', 'pos']
        sents = sents.reset_index().drop('index', axis=1)
        not_s_articles = not_s_articles.reset_index().drop('index', axis=1)
        not_s_articles = pd.concat([not_s_articles, sents], axis=1)
        s_articles = pd.concat([s_articles, not_s_articles], axis=0)
        s_articles.to_csv('datasets/articles.csv', encoding='utf8')
        print 'updated sentiments'
        return s_articles
    
    else:
        print 'Nothing updated'
        return s_articles
    
def get_sentimentaized_articles(companies):
    articles = sn_data.get_articles(companies)
    return update_sentimentalized_articles(articles)

### need to work on this so it can use multi part files

In [ ]:
# all_articles = get_sentimentaized_articles(companies)

In [ ]:
all_articles.info()

In [ ]:
def get_quote(symbol, start_date='1/1/2014', end_date=date.today().strftime('%m/%d/%Y')):
    symbol = symbol.replace('.N', '')
    try:
        sleep(2)
        ret = data.DataReader(symbol, data_source='yahoo', start=start_date, end=end_date)
    except (urllib2.URLError, IOError) as e:
        print symbol, e.args
        sleep(10)
        ret = data.DataReader(symbol, data_source='google', start=start_date, end=end_date)
        ret['Adj Close'] = None
        pass
    ret['Symbol'] = symbol
    return ret

def update_quotes(articles, reset=False):
    not_downloaded = False
    quotes = False
    if not os.path.exists('datasets/daily_quotes.csv'):
        not_downloaded = list(articles.Symbol.unique())
        print 'Its empty!'
    else:
        quotes = pd.read_csv('datasets/daily_quotes.csv', encoding='utf8', index_col=0)
        not_downloaded = [x for x in list(articles.Symbol.unique()) if x not in list(quotes.Symbol.unique())]
        print "Available symbol quotes: ", len(list(quotes.Symbol.unique()))
    
    for symbol in not_downloaded:
        quote = get_quote(symbol)
        if not quotes.empty:
            quotes = pd.concat([quotes, quote], axis=0)
        else:
            quotes = quote
        quotes.to_csv('datasets/daily_quotes.csv', encoding='utf8')
        
    print "New downloaded symbol quotes: ", len(not_downloaded)
    quotes = pd.read_csv('datasets/daily_quotes.csv', encoding='utf8')
    return quotes
    
    

In [ ]:
quotes = update_quotes(all_articles)

### We may have some problems with these two: `BRKB` and `BFB`

In [ ]:
all_articles.shape

In [ ]:
all_articles.Time = all_articles.Time.apply(lambda x: re.search('[0-9]?[0-9]:[0-9]{1,2}', x).group(0))
all_articles['Date'] = pd.to_datetime(all_articles['Date'], format='%Y-%m-%d')

In [ ]:
quotes = quotes.drop('Adj Close', axis=1)
quotes['Date'] = pd.to_datetime(quotes['Date'], format='%Y-%m-%d')
quotes.head()

In [ ]:
articles = all_articles.copy()
articles = articles[['Date', 'Symbol', 'compound', 'neg', 'neu', 'pos']]

In [ ]:
articles = pd.pivot_table(articles,
                          index=['Date', 'Symbol'],
                          values=['compound', 'neg', 'neu', 'pos'],
                          aggfunc=np.average).reset_index()

In [ ]:
articles.head()

In [ ]:
print len(articles['Symbol'].unique()), len(quotes['Symbol'].unique())

In [ ]:
print articles.shape, quotes.shape

In [ ]:
quotes_articles = quotes.set_index(['Date', 'Symbol']).join(
    articles.set_index(['Date', 'Symbol']), rsuffix='r').reset_index()

In [ ]:
quotes_articles.head()

In [ ]:
quotes_articles['Next_Close'] = quotes_articles['Close'].shift(-1)
quotes_articles['Next_Open'] = quotes_articles['Open'].shift(-1) # for some possible prediction
quotes_articles['Prev_Close'] = quotes_articles['Close'].shift(1)

In [ ]:
quotes_articles.head()

### This code needs to be cleaned. In first quote of symbols Prev_Close should be NaN also in last quote of symbol Next_close should be NaN

In [ ]:
for symbol in quotes_articles['Symbol'].unique():
    min_date = quotes_articles[(quotes_articles['Symbol'] == symbol)].Date.min()
    max_date = quotes_articles[(quotes_articles['Symbol'] == symbol)].Date.max()
    quotes_articles.Prev_Close[(quotes_articles['Symbol'] == symbol) & (quotes_articles['Date'] == min_date)] = None
    quotes_articles.Prev_Close[(quotes_articles['Symbol'] == symbol) & (quotes_articles['Date'] == max_date)] = None

In [ ]:
quotes_articles.head()

In [ ]:
slopesfunc = lambda x: pd.Series([
        x['Close'] - x['Prev_Close'],
        x['Next_Close'] - x['Close'],
        (x['Next_Close'] - x['Close']) - (x['Close'] - x['Prev_Close']),
    ])

In [ ]:
tmp = quotes_articles.apply(slopesfunc, axis=1)
tmp.columns = ['Prev_Slope', 'Next_Slope', 'Diff_Slope']

In [ ]:
quotes_articles = pd.concat([quotes_articles, tmp], axis=1)

In [ ]:
has_news = quotes_articles[(quotes_articles['compound'].notnull())]
has_news.head()

In [ ]:
def get_cv(target):
    return StratifiedKFold(target, n_folds=3, shuffle=True, random_state=41)

def score(model, data, target):
    model.fit(data, target)
    
    return model.score(data, target)
def grid_search(model, params, cv):
    return GridSearchCV(estimator=model, 
                    param_grid=params,
                    cv=cv
                   )
def check_regression_results(y_true, y_pred):
    print "Explained variance regression score: ", explained_variance_score(y_true, y_pred, multioutput='uniform_average')
    print "Mean Absolute Error:", mean_absolute_error(y_true, y_pred)

def draw_feature_importance(model, data):
    importances = model.feature_importances_
    std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)
    indices = np.argsort(importances)[::-1]
    feature_names = data.columns
    # Plot the feature importances of the model
    plt.figure(figsize=(20,15))
    plt.title("Feature importances")
    plt.bar(range(data.shape[1]), importances[indices],
            color="r", yerr=std[indices], align="center")
    plt.xticks(range(data.shape[1]), feature_names[indices], rotation=90)
    plt.xlim([-1, data.shape[1]])
    plt.show()

def model_test_metrics(model, data, target):
    print "Test Data score: ", model.fit(data, target).score(data, target)
    check_regression_results(target, model.predict(data))

def evaluate_model(model, data, target, params=None, draw_features=True, verbose=True):
    train_data, test_data, train_target, test_target=train_test_split(data, target)
#     print score(model, train_data, train_target), "\n"
    cv=get_cv(train_target)
    if params:
        grid=grid_search(model, params, cv)
    
        grid.fit(train_data, train_target)
        model = grid.best_estimator_
        if verbose:
            print "Best Model after Grid Search:\n", model
        
    else:
        model.fit(train_data, train_target)
        
    s=cross_val_score(model, train_data, train_target, cv=cv, n_jobs=-1)
    if verbose:
        print "Mean score of the model is: {}".format(s.mean())
        predictions = model.predict(test_data)
        check_regression_results(test_target, predictions)

    if draw_features:
        draw_feature_importance(model, train_data)
    return model

In [ ]:
plt.figure(figsize=(25,15))
sns.heatmap(has_news.corr(), vmax=.3, square=True,
            linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
has_news = has_news.dropna()

In [ ]:
x = has_news[['Close', 'High', 'Low', 'Open', 'Volume', 'compound', 'neg', 'neu', 'pos', 'Next_Open', 'Prev_Slope']]
y = has_news['Diff_Slope']

In [ ]:
plt.figure(figsize=(25,15))
sns.heatmap(x.corr(), vmax=.3, square=True,
            linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.3)

In [ ]:
# params = {
#     'n_estimators': [10, 20, 50, 100, 200],
#     'max_depth': [None,1, 2, 3, 4, 5],
#     'min_samples_split': [2, 10, 25, 50, 100],
# #     'min_samples_leaf': [1, 2, 3, 4, 5, 10]
# }

# %time lr = evaluate_model(LinearRegression(n_jobs=-1), x_train, y_train, draw_features=False)
lr = evaluate_model(LinearRegression(n_jobs=-1), x_train, y_train, draw_features=False)

In [ ]:
lr_predictions = lr.predict(x_test)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_test, lr_predictions)
ax.plot([lr_predictions.min(), lr_predictions.max()], [lr_predictions.min(), lr_predictions.max()], 'k--', lw=2)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.figure(figsize=(1,1))
plt.show()

In [ ]:
%time dt=evaluate_model(DecisionTreeRegressor(), x_train, y_train, draw_features=False)

In [ ]:
dt_predictions = dt.predict(x_test)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_test, dt_predictions)
ax.plot([dt_predictions.min(), dt_predictions.max()], [dt_predictions.min(), dt_predictions.max()], 'k--', lw=2)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.figure(figsize=(1,1))
plt.show()

In [ ]:
%time rf=evaluate_model(RandomForestRegressor(n_jobs=-1), x_train, y_train)

In [ ]:
rf_predictions = rf.predict(x_test)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_test, rf_predictions)
ax.plot([rf_predictions.min(), rf_predictions.max()], [rf_predictions.min(), rf_predictions.max()], 'k--', lw=2)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.figure(figsize=(20,15))
plt.show()

In [ ]:
%time et=evaluate_model(ExtraTreesRegressor(n_jobs=-1), x_train, y_train)

In [ ]:
et_predictions = et.predict(x_test)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_test, et_predictions)
ax.plot([et_predictions.min(), et_predictions.max()], [et_predictions.min(), et_predictions.max()], 'k--', lw=2)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.figure(figsize=(20,15))
plt.show()

In [ ]:
# params = {
#     'n_estimators': [10, 20, 50, 100, 200],
#     'max_depth': [None,1, 2, 3, 4, 5],
#     'min_samples_split': [2, 10, 25, 50, 100],
# #     'min_samples_leaf': [1, 2, 3, 4, 5, 10]
# }
# start_time = time.time()
# rf=evaluate_model(RandomForestRegressor(n_jobs=-1), x_train, y_train, params)
# print 'This operation took '+ str(time.time() - start_time) + " seconds"